In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
path = 'drive/MyDrive/meme'


In [4]:
event = pd.read_csv(path+'/danila_event.csv')


In [7]:
meme = pd.read_csv(path+'/danila_meme.csv')
user = pd.read_csv(path+'/danila_user.csv')

In [9]:
meme.head()

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0
1,10873,photo,43.0,2020-03-25 11:48:14+00,NaN,ru,True,79,74,2020-03-25 20:00:01.154096+00,2020-10-26 19:20:58.226846+00,False,"Я не робот Докажите, что всякое односвязное Ит...",ww,2.0
2,9874,photo,78.0,2020-03-24 07:18:03+00,Anime toddies expert,en,True,1,7,2020-03-24 13:06:18.588359+00,2020-10-26 19:20:58.259045+00,False,When you find the actual interview and it does...,en,3.0
3,43458,photo,43.0,2020-05-23 12:50:52+00,NaN,ru,False,8,12,2020-05-24 22:03:34.926116+00,2020-10-26 19:20:58.265179+00,False,Google сколько калорий в грамме урана а Все В ...,ww,2.0
4,9000,photo,76.0,2020-03-23 07:16:05+00,New treasure,und,True,2,9,2020-03-23 13:04:55.258554+00,2020-10-26 19:20:58.29742+00,False,YRO .20.,en,3.0


In [10]:
user.head()

,user_id,deep_link,is_blocked_bot,is_banned,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender
0,820326025,anime0ch_lang_ru,False,False,False,6,1,20,2020-09-12 20:07:11.668743+00,2020-10-26 12:16:18.458654+00,ru,M
1,1040875164,likemeupbot_lang_en,False,False,False,2,1,7,2020-07-21 05:54:12.671036+00,2020-10-26 14:37:05.637591+00,en,NaN
2,449432539,debil_ebaniy,True,False,False,2,1,9,2020-09-14 19:02:06.80911+00,2020-10-17 21:26:52.029824+00,ru,F
3,230248850,NaN,False,False,False,26,1,38,2020-03-16 00:36:42.252438+00,2020-10-26 12:11:02.159405+00,ru,M
4,7719424,NaN,True,False,False,27,6,49,2020-03-15 22:30:45.300242+00,2020-10-17 21:26:52.19941+00,ru,M


In [11]:
event.head()

,event_id,reaction_id,session_idx,meme_idx,created_at,updated_at,meme_id,user_id
0,3607617,2.0,55,11,2020-10-24 19:45:50.964474+00,2020-10-24 19:45:51.049752+00,24379,280747086
1,3608100,1.0,280,45,2020-10-24 19:52:33.681933+00,2020-10-24 19:52:33.763993+00,49173,285739883
2,3607629,1.0,55,13,2020-10-24 19:46:08.690407+00,2020-10-24 19:46:08.779079+00,8340,280747086
3,3607649,NaN,30,2,2020-10-24 19:46:24.443014+00,2020-10-24 19:46:24.519787+00,64748,78991522
4,3608045,2.0,35,341,2020-10-24 19:51:28.544856+00,2020-10-24 19:51:28.586788+00,45163,860086058


In [12]:
# Создать рекомендательную систему для каждого пользователя

In [18]:
#meme['is_broken'].value_counts()

meme = meme[meme['is_broken'] == False]

In [19]:
meme

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0
1,10873,photo,43.0,2020-03-25 11:48:14+00,NaN,ru,True,79,74,2020-03-25 20:00:01.154096+00,2020-10-26 19:20:58.226846+00,False,"Я не робот Докажите, что всякое односвязное Ит...",ww,2.0
2,9874,photo,78.0,2020-03-24 07:18:03+00,Anime toddies expert,en,True,1,7,2020-03-24 13:06:18.588359+00,2020-10-26 19:20:58.259045+00,False,When you find the actual interview and it does...,en,3.0
3,43458,photo,43.0,2020-05-23 12:50:52+00,NaN,ru,False,8,12,2020-05-24 22:03:34.926116+00,2020-10-26 19:20:58.265179+00,False,Google сколько калорий в грамме урана а Все В ...,ww,2.0
4,9000,photo,76.0,2020-03-23 07:16:05+00,New treasure,und,True,2,9,2020-03-23 13:04:55.258554+00,2020-10-26 19:20:58.29742+00,False,YRO .20.,en,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61851,29440,photo,80.0,2020-04-12 16:52:08+00,She never needed makeup,en,True,2,8,2020-04-12 21:02:06.757845+00,2020-10-26 19:20:58.103871+00,False,When your girlfriend takes off her makeup but ...,en,3.0
61852,50912,photo,18.0,2020-07-18 17:00:07+00,NaN,ru,True,539,225,2020-07-19 06:14:42.821713+00,2020-10-26 19:20:58.113269+00,False,katevictoriax Парней нужно чаще ласкать. Клади...,ww,2.0
61853,22646,photo,2.0,2020-04-06 06:52:16+00,NaN,en,False,1,2,2020-04-06 08:00:00.352386+00,2020-10-26 19:20:58.143725+00,False,I COME TOTHEDARKSIDE WE HAVE RESPIRATOR MASKS,ww,3.0
61854,49822,photo,NaN,2020-07-10 12:08:55+00,NaN,ru,True,5,17,2020-07-10 18:15:20.413744+00,2020-10-26 19:20:58.148913+00,False,"Шаг Не знаете, куда поехать? Едьте в центр! Ян...",ww,2.0


In [21]:
user['is_banned'].value_counts()

False    19918
Name: is_banned, dtype: int64

In [25]:
# Проверить гипотезу: англ пользователи более активные чем русские
# лайки
# дизлайки
# отношение L/D
# Диаграмма (рус в среднем и eng в среднем)

In [26]:
user['interface_lang'].value_counts()

ru    14156
en     5714
Name: interface_lang, dtype: int64

In [63]:
reactions = event.groupby('user_id').count()['event_id']
reactions

user_id
1615           47
3859          193
12199           3
13934           6
14229          15
             ... 
1399229617      1
1399329801      1
1399419434     11
1399511615      2
1399599499      9
Name: event_id, Length: 19699, dtype: int64

In [64]:
# 1 - лайк
# 2 - дизлайк
likes = event[event['reaction_id'] == 1].groupby('user_id').count()['event_id']
#likes.rename(columns = {'event_id':'likes'})
likes


user_id
1615           25
3859          106
13934           1
14229           2
16060          14
             ... 
1398188580      5
1398529159     12
1398799657      6
1399419434      6
1399599499      8
Name: event_id, Length: 16633, dtype: int64

In [65]:
dislikes =  event[event['reaction_id'] == 2].groupby('user_id').count()['event_id']
dislikes


user_id
1615           11
3859           83
14229          10
16060         111
20535          10
             ... 
1397824939      2
1397992921      5
1397993276      3
1398022998     14
1398799657      1
Name: event_id, Length: 15881, dtype: int64

In [55]:
event_reactions = pd.DataFrame([likes['reaction_id'].values,dislikes['reaction_id'].values, reactions['reaction_id'].values]).transpose()
event_reactions.rename(columns = {0:'likes', 1:'dislikes', 2:'reactions'}, inplace = True)

In [58]:
event_reactions.index = reactions.index
event_reactions

,likes,dislikes,reactions
user_id,,,
1615,25.0,11.0,46.0
3859,106.0,83.0,190.0
12199,1.0,10.0,2.0
13934,2.0,111.0,5.0
14229,14.0,10.0,14.0
...,...,...,...
1399229617,NaN,NaN,0.0
1399329801,NaN,NaN,0.0
1399419434,NaN,NaN,10.0


In [69]:
likes_users_ids = likes.index
likes_users_ids

Int64Index([      1615,       3859,      13934,      14229,      16060,
                 20535,      28327,      30767,      37459,      39129,
            ...
            1397540701, 1397824939, 1397992921, 1397993276, 1398022998,
            1398188580, 1398529159, 1398799657, 1399419434, 1399599499],
           dtype='int64', name='user_id', length=16633)

In [76]:
users_likes = user[user.user_id.isin(likes_users_ids)].set_index('user_id')['interface_lang'].copy()
users_likes

user_id
820326025     ru
1040875164    en
449432539     ru
1008072970    en
8239465       ru
              ..
125474047     ru
608727447     ru
1254871249    ru
184147840     ru
761418142     ru
Name: interface_lang, Length: 16633, dtype: object

In [84]:
detailed_likes =  pd.DataFrame([users_likes, likes]).transpose()
grouped_det_likes = detailed_likes.groupby('interface_lang').count()
grouped_det_likes

,event_id
interface_lang,
en,4546
ru,12087


In [86]:
import plotly.express as px
fig = px.bar(grouped_det_likes, x = grouped_det_likes.index, y = 'event_id')
fig.show()

In [87]:
# Для дизлайков и всех реакций (ДЗ)

In [88]:
# Для каждого source соотношение лайков дизлайков

In [91]:
memes_grouped = meme[['source_id', 'likes', 'dislikes']]
memes_grouped_sum = memes_grouped.groupby('source_id').sum()
memes_grouped_sum['l\\d'] = memes_grouped_sum['likes'] / memes_grouped_sum['dislikes']
memes_grouped_sum


,likes,dislikes,l\d
source_id,,,
1.0,141,447,0.315436
2.0,8753,11876,0.737033
3.0,17852,19303,0.924830
4.0,17265,27841,0.620129
5.0,11334,15323,0.739672
...,...,...,...
155.0,172,463,0.371490
156.0,741,1331,0.556724
157.0,823,1532,0.537206


# ДЗ
- Построить гипотезы (2-3 штуки)
- Датасет + модель (какой мем какому пользователю зайдет)
- Реком система в цикле(как вариант) будет смотреть каждый мем и классифицировать на 1 или 0
- Презентация